### Question 1: Build, Run and Test Docker Image

In [29]:
import os

# Build the Docker image
!docker build -f Dockerfile_full -t zoomcamp-model:3.13.10-hw10 .

DEPRECATED: The legacy builder is deprecated and will be removed in a future release.
            Install the buildx component to build images with BuildKit:
            https://docs.docker.com/go/buildx/

Sending build context to Docker daemon  983.9MB
Step 1/10 : FROM python:3.13.10-slim-bookworm
 ---> b6cd69eec6b2
Step 2/10 : COPY --from=ghcr.io/astral-sh/uv:latest /uv /uvx /bin/
 ---> Using cache
 ---> cbae5dac01c4
Step 3/10 : ENV PATH="/code/.venv/bin:$PATH"
 ---> Using cache
 ---> 59c5bf62a601
Step 4/10 : WORKDIR /code
 ---> Using cache
 ---> a8c80dfdcda3
Step 5/10 : COPY "pyproject.toml" "uv.lock" ".python-version" ./
 ---> Using cache
 ---> 557292f7289f
Step 6/10 : RUN uv sync --locked
 ---> Using cache
 ---> d34057fcba29
Step 7/10 : COPY pipeline_v2.bin .
 ---> Using cache
 ---> 80b39114869d
Step 8/10 : COPY "q6_predict.py" ./
 ---> Using cache
 ---> f6134e8fe5fb
Step 9/10 : EXPOSE 9696
 ---> Using cache
 ---> d968b9fbb964
Step 10/10 : ENTRYPOINT ["uvicorn", "q6_predict:app", 

In [30]:
print("Now, run the Docker image locally and start the model server")
!docker run -it --rm -p 9696:9696 zoomcamp-model:3.13.10-hw10

Now, run the Docker image locally and start the model server
INFO:     Started server process [1]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:9696 (Press CTRL+C to quit)
^C
INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [1]


In [32]:
print("We observe the output to find the 'conversion_probability' value")
!python q6_test.py

We observe the output to find the 'conversion_probability' value
{'conversion_probability': 0.49999999999842815, 'conversion': False}


### Question 2: kind Version

In [33]:
# We execute to find the kind version
!kind --version

kind version 0.30.0


### Question 3: Smallest Deployable Unit

### Answer
In Kubernetes, the smallest deployable computing unit that can be created and managed is a **Pod**.

*   **Node**: A Node is a worker machine in Kubernetes, which can be a virtual or physical machine. It hosts Pods.
*   **Pod**: A Pod is the smallest and simplest unit in the Kubernetes object model that you create or deploy. A Pod represents a single instance of a running process in your cluster, and it typically encapsulates one or more containers (such as Docker containers), storage resources, a unique network IP, and options that govern how the container(s) should run.
*   **Deployment**: A Deployment is a higher-level object that manages stateless applications. It ensures that a specified number of Pod replicas are running at any given time. While Deployments manage Pods, the Pods themselves are the actual deployable units.
*   **Service**: A Service is an abstract way to expose an application running on a set of Pods as a network service. It enables network access to a group of Pods.

Therefore, the correct answer is **Pod**.

### Question 4: Get Kubernetes Service

In [36]:
print("First, we create a Kubernetes cluster using kind.")
!kind create cluster

First, we create a Kubernetes cluster using kind.
Creating cluster "kind" ...
 ✓ Ensuring node image (kindest/node:v1.34.0) 🖼
 ✓ Preparing nodes 📦 7l
 ✓ Writing configuration 📜
 ✓ Starting control-plane 🕹️7l
 ✓ Installing CNI 🔌7l
 ✓ Installing StorageClass 💾7l
Set kubectl context to "kind-kind"
You can now use your cluster with:

kubectl cluster-info --context kind-kind

Have a nice day! 👋


In [37]:
print("Now, we verify that the cluster was successfully created.")
!kubectl cluster-info

Now, we verify that the cluster was successfully created.
Kubernetes control plane is running at https://127.0.0.1:38433
CoreDNS is running at https://127.0.0.1:38433/api/v1/namespaces/kube-system/services/kube-dns:dns/proxy

To further debug and diagnose cluster problems, use 'kubectl cluster-info dump'.


In [38]:
print("Finally, we list the services currently running in Kubernetes cluster")
!kubectl get services
print("We observe the 'Type' column to find the answer for Question 4.")

Finally, we list the services currently running in Kubernetes cluster
NAME         TYPE        CLUSTER-IP   EXTERNAL-IP   PORT(S)   AGE
kubernetes   ClusterIP   10.96.0.1    <none>        443/TCP   16s
We observe the 'Type' column to find the answer for Question 4.


### Question 5: Load Docker Image into kind

In [39]:
print("We load the Docker image 'zoomcamp-model:3.13.10-hw10' into kind cluster")
!kind load docker-image zoomcamp-model:3.13.10-hw10

We load the Docker image 'zoomcamp-model:3.13.10-hw10' into kind cluster
Image: "zoomcamp-model:3.13.10-hw10" with ID "sha256:a3a6661a77b839802cfd609e557f8eb481d8dc994ea55362c7af27412a099b69" not yet present on node "kind-control-plane", loading...


### Question 6: Create Kubernetes Deployment

#### Deployment Configuration for Question 6

This is the `deployment.yaml` content with the placeholders filled in:

*   `<Image>`: `zoomcamp-model:3.13.10-hw10` 
*   `<Memory>`: `256Mi`
*   `<CPU>`: `200m`
*   `<Port>`: `9696`

We create a file named `deployment.yaml` and add the following content to it:

```yaml
apiVersion: apps/v1
kind: Deployment
metadata:
  name: subscription
spec:
  selector:
    matchLabels:
      app: subscription
  replicas: 1
  template:
    metadata:
      labels:
        app: subscription
    spec:
      containers:
      - name: subscription
        image: zoomcamp-model:3.13.10-hw10
        resources:
          requests:
            memory: "64Mi"
            cpu: "100m"            
          limits:
            memory: 256Mi
            cpu: 200m
        ports:
        - containerPort: 9696
```


In [40]:
print("We save the `deployment.yaml` content. Then, apply the deployment to Kubernetes cluster")
!kubectl apply -f deployment.yaml

We save the `deployment.yaml` content. Then, apply the deployment to Kubernetes cluster
deployment.apps/subscription created


**Reasoning**:
After applying the deployment, the next step is to verify that the Pods have been created and are running as expected, as instructed in the subtask.



In [41]:
print("Now, we verify that the deployment has created a running Pod")
!kubectl get pods

Now, we verify that the deployment has created a running Pod
NAME                            READY   STATUS    RESTARTS   AGE
subscription-5bc7fdcf85-c2whg   1/1     Running   0          5s


### Analyze Question 7: Create Kubernetes Service

#### Service Configuration for Question 7

Based on the instructions and the `deployment.yaml` from Question 6, this is the `service.yaml` content with the placeholders filled in:

*   `<Service name>`: `subscription-service` (a descriptive name for the service)
*   `<???>` (for `selector.app`): `subscription` (this must match the `app` label in our `deployment.yaml`)
*   `<PORT>` (for `targetPort`): `9696` (this must match the `containerPort` from our `deployment.yaml`)

We create a file named `service.yaml` and add the following content to it:

```yaml
apiVersion: v1
kind: Service
metadata:
  name: subscription-service
spec:
  type: LoadBalancer
  selector:
    app: subscription
  ports:
  - port: 80
    targetPort: 9696
```

In [42]:
print("We save the `service.yaml` content provided in the previous step to a file named `service.yaml`.")
print("Then, we apply the service to your Kubernetes cluster:")
!kubectl apply -f service.yaml

We save the `service.yaml` content provided in the previous step to a file named `service.yaml`.
Then, we apply the service to your Kubernetes cluster:
service/subscription-service created


In [43]:
print("Now, we verify that the service has been created")
!kubectl get services

Now, we verify that the service has been created
NAME                   TYPE           CLUSTER-IP      EXTERNAL-IP   PORT(S)        AGE
kubernetes             ClusterIP      10.96.0.1       <none>        443/TCP        68s
subscription-service   LoadBalancer   10.96.203.249   <pending>     80:31216/TCP   4s
